In [1]:
import os

In [2]:
%pwd

'c:\\Users\\user\\Desktop\\E2E project\\ML-wine-project-End-to-end\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\user\\Desktop\\E2E project\\ML-wine-project-End-to-end'

In [6]:
# Creating an entity

from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    Source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [16]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories

In [17]:

# for src configuration
class ConfigurationManager:
    def __init__(self, 
                 config_pathway=CONFIG_FILE_PATH, 
                 param_pathway=PARAM_FILE_PATH, 
                 schema_pathway=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_pathway)
        self.param = read_yaml(param_pathway)
        self.schema = read_yaml(schema_pathway)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            Source_URL= config.Source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [18]:
#data ingestion component

import os
import zipfile
import urllib.request as request
from src.mlproject.logging import logger
from src.mlproject.utils.common import get_size



In [19]:
# Data ingestion configuration

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.Source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f'{filename} downloaded! with following info \n{headers}')

        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: zip
        extract zip file into the directory
        function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
# pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-12-24 09:56:16,163: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-24 09:56:16,173: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-24 09:56:16,188: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-24 09:56:16,191: INFO: common: created directory at: artifacts]
[2024-12-24 09:56:16,192: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-24 09:56:17,554: INFO: 2865268503: artifacts/data_ingestion/data.zip downloaded! with following info 
Connection: close
Content-Length: 21984
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "85a38be3d4c70e3b6dc97fc47873623da88c9b850e1526aeb57ff236bdd2815c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0BF1:271291:4AD09D:64829D:676A8540
Accept-Ranges: bytes
Date: Tue, 2